Passo a passo:
1. Receber os logs e modelos como na v1
2. Converter o log de atividades para o novo formato, passando os acesso para o log de atividades
4. Adaptar o modelo declare para que todas as operações obrigatórias e proibidas do modelo de acesso virem regras declare por meio das restrições de ativação
5. Colocar a restrição de recurso como regra de correlação

**Parei porque percebi que não tinha como passar uma lista de recursos como parâmetro e era inviável deixar tudo hardcoded, além disso, é necessário ver como verificar o recurso para todas as atividades existentes no log**

In [121]:
from Declare4Py.ProcessModels.DeclareModel import DeclareModel

def check_letters(cell, modelo, acesso, atividade):
    """Verifica quais letras (c, r, u, d) estão presentes na célula, diferenciando maiúsculas e minúsculas."""
    # Letras a serem verificadas
    letters = ['c', 'r', 'u', 'd']
    
    if atividade == 'Ferramenta':
        return modelo
    
    cell = str(cell)  # Garantir que a célula é string
    for letter in letters:
        uppercase_present = letter.upper() in cell
        
        if uppercase_present:
            if f'{acesso} {letter}\n' not in modelo:
                modelo += 'activity ' + f'{acesso} {letter}\n'
            modelo += f'Precedence[{atividade} begin, {acesso} {letter}] | A.concept:resource is re7 | T.concept:resource is re7 |\n'
            modelo += f'Response[{acesso} {letter}, {atividade} complete] | A.concept:resource is re7 | T.concept:resource is re7 |\n'
    
    return modelo
    

def convertModelToRules(modeloAcesso, modeloProcesso):
    declare_model_activities = modeloProcesso.activities
    declare_model_constraints = modeloProcesso.serialized_constraints
    
    novoModelo = ''
    
    for act in declare_model_activities:
        novoModelo += 'activity ' + act + ' begin' + '\n'
        novoModelo += 'activity ' + act + ' complete' +'\n'

    # Criar um novo DataFrame com os resultados
    for col in modeloAcesso.columns:
        for index, value in modeloAcesso[col].items():
            novoModelo = check_letters(value, novoModelo, modeloAcesso.iloc[index,0], col)
            
    for rule in declare_model_constraints:
        if ',' in rule:
            rule1, rule2 = rule.split(',')
            rule2, rule3 = rule2.split(']')
            novoModelo += rule1 + ' begin,' + rule2 + ' begin' + ']' + rule3 + '\n'
        else:
            rule1, rule2 = rule.split(']')
            novoModelo += rule1 + ' begin' + ']' + rule2 + '\n'
    declare_model = DeclareModel().parse_from_string(novoModelo)
    declare_model.to_file('modeloRegrasConjuntov4.decl')
    return declare_model

In [122]:
import pandas as pd
from Declare4Py.ProcessModels.DeclareModel import DeclareModel
from Declare4Py.D4PyEventLog import D4PyEventLog
import pm4py

def PreProcessData(logProcessoPATH, logAcessoPATH, modeloRecursoPATH, modeloProcessoPATH, modeloAcessoPATH):
    logAcesso = D4PyEventLog(case_name="case:concept:name")
    logAcesso.parse_xes_log(logAcessoPATH)
    logAcessoProcessado = pm4py.convert_to_dataframe(logAcesso.get_log())
    logAcessoProcessado = logAcessoProcessado.sort_values(['case:concept:name', 'concept:instance'])
    modeloRecursoProcessado = pd.read_csv(modeloRecursoPATH, sep=';')
    modeloAcessoProcessado = pd.read_csv(modeloAcessoPATH, sep=';')
    declare_model = DeclareModel().parse_from_file(modeloProcessoPATH)
    event_log = D4PyEventLog(case_name="case:concept:name")
    event_log.parse_xes_log(logProcessoPATH)
    return event_log, logAcessoProcessado, modeloRecursoProcessado, declare_model, modeloAcessoProcessado
    

In [123]:
import pm4py

def convertLogs(logProcesso, logAcesso):
    '''
    Converte os logs de processo e acesso em um log único que pode ser submetido a verificação de conformidade 
    por meio da biblioteca Declare4Py 
    '''
    logProcessoCSV = pm4py.convert_to_dataframe(logProcesso.get_log())
    logProcessoCSV = logProcessoCSV.sort_values(['case:concept:name', 'concept:instance'])
    for index, row in logProcessoCSV.iterrows():
        logProcessoCSV.at[index, 'concept:name'] = row['concept:name'] + ' ' + row['lifecycle:transition']
    logProcessoCSV['lifecycle:transition'] = 'complete'
    
    for index, row in logAcesso.iterrows():
        logProcessoCSV.loc[len(logProcessoCSV)] = [(row['concept:tool'] + ' ' + row['concept:operation'].lower()), 'complete' ,row['time:timestamp'], row['concept:resource'], row['concept:instance'], len(logProcessoCSV), row['@@case_index'], row['case:concept:name']]
    logProcessoCSV = logProcessoCSV.sort_values(['case:concept:name', 'concept:instance','time:timestamp'])
    logProcessoCSV.to_csv('LogTesteConjuntov4.csv', index=False)
    pm4py.write_xes(logProcessoCSV, "LogSinteticoConjuntoOFICIALv4.xes")

In [124]:
def formatViolations(df_violations):
    '''
    Formata as violações encontradas na verificação de conformidade do Declare4Py
    '''
    violacoes = []
    for index, row in df_violations.iterrows():
        for coluna in df_violations.columns:
            if row[coluna] == 1:
                violacoes.append(f"Trace {index} violou {coluna}")
    return violacoes

In [125]:
from Declare4Py.ProcessMiningTasks.ConformanceChecking.MPDeclareAnalyzer import MPDeclareAnalyzer
from Declare4Py.ProcessMiningTasks.ConformanceChecking.MPDeclareResultsBrowser import MPDeclareResultsBrowser

def CheckProcessConformance(modeloProcesso):
    '''
    Checa conformidade de processo entre o log e o modelo DECLARE com a biblioteca Declare4Py
    Recebe um log de processo (em formato EventLog) e um modelo de processo (em DeclareModel)
    
    '''
    event_log = D4PyEventLog(case_name="case:concept:name")
    event_log.parse_xes_log("LogSinteticoConjuntoOFICIALv4.xes")
    
    model_constraints = modeloProcesso.get_decl_model_constraints()

    # print("Model constraints:")
    # print("-----------------")
    # for idx, constr in enumerate(model_constraints):
    #     print(idx, constr)
    basic_checker = MPDeclareAnalyzer(log=event_log, declare_model=modeloProcesso, consider_vacuity=False)
    conf_check_res: MPDeclareResultsBrowser = basic_checker.run()
    violacoes = formatViolations(conf_check_res.get_metric(metric="num_violations"))
    return violacoes

In [126]:
def manter_maiusculas(valor):
    if isinstance(valor, str):
        return ''.join([c for c in valor if c.isupper()])
    return ''

def CheckAcessConformance(logProcesso,logAcesso, modeloAcesso):
    '''
    Checa conformidade entre o log de acesso, log de processo e o modelo de acesso
    Recebe modelo de acesso e logs em csv
    '''
    logProcessoCSV = pm4py.convert_to_dataframe(logProcesso.get_log())
    logProcessoCSV = logProcessoCSV.sort_values(['case:concept:name', 'concept:instance'])
    violacoes = {}
    violacoes['OperacaoProibida'] = []
    violacoes['AcessoObgNaoRealizado'] = []
    logProcessoFiltrado = logProcessoCSV[logProcessoCSV["lifecycle:transition"] == "begin"]
    logAcessoObg = modeloAcesso.copy()
    logAcessoObg.loc[:, logAcessoObg.columns != "Ferramenta"] = logAcessoObg.loc[:, logAcessoObg.columns != "Ferramenta"].map(manter_maiusculas)
    for index, row in logProcessoFiltrado.iterrows():
        demanda = row['case:concept:name']
        atividade = row['concept:instance']
        nome = row['concept:name']
        acessos = logAcesso[(logAcesso['case:concept:name'] == demanda) & (logAcesso['concept:instance'] == atividade)]
        modeloAtv = modeloAcesso[['Ferramenta', nome]]
        modeloAtvObg = logAcessoObg[['Ferramenta', nome]].copy()
        modeloAtvObg.loc[:,'Realizada'] = modeloAtvObg[nome].apply(lambda x: False if (isinstance(x, str) and len(x) > 0) else True)
        for i, acc in acessos.iterrows():
            ferramenta = acc['concept:tool']
            opPermitidas = modeloAtv[modeloAtv["Ferramenta"] == ferramenta][nome].iloc[0].split(', ') if isinstance(modeloAtv[modeloAtv["Ferramenta"] == ferramenta][nome].iloc[0], str) else []
            if acc['concept:operation'] not in opPermitidas:
                violacoes['OperacaoProibida'].append([row["concept:name"], demanda, acc['concept:operation'], ferramenta])
            if acc['concept:operation'].lower() == modeloAtvObg[modeloAtvObg["Ferramenta"] == ferramenta][nome].iloc[0].lower():
                modeloAtvObg.loc[modeloAtvObg["Ferramenta"] == ferramenta, "Realizada"] = True
        if not modeloAtvObg['Realizada'].all():
            naoRealizados = modeloAtvObg.loc[modeloAtvObg['Realizada'] == False, 'Ferramenta']     
            violacoes['AcessoObgNaoRealizado'].append([row["concept:name"], demanda, ', '.join(naoRealizados.tolist())])
    return violacoes
    

In [127]:
import pm4py

def CheckResourceConformance(logProcesso, logAcesso, modeloRecurso):
    '''
    Checa conformidade entre o log de acesso, log de processo e o modelo de recurso
    Recebe modelo de recurso e logs em csv
    '''
    
    logProcessoCSV = pm4py.convert_to_dataframe(logProcesso.get_log())
    logProcessoCSV = logProcessoCSV.sort_values(['case:concept:name', 'concept:instance'])
    logProcessoFiltrado = logProcessoCSV[logProcessoCSV["lifecycle:transition"] == "begin"]
    
    violacoes = {}
    violacoes["AcessoRecursoEquipe"] = []
    violacoes["AcessoRecursoErrado"] = []
    violacoes["Atividade"] = []
    for index, row in modeloRecurso.iterrows():
        demanda = row['case:concept:name']
        recursos = row['concept:resources'].split(", ")
        atividades = logProcessoFiltrado[logProcessoFiltrado['case:concept:name'] == demanda]
        acessos = logAcesso[logAcesso['case:concept:name'] == demanda]
        for i, atv in atividades.iterrows():
            recursoAtv = atv['concept:resource']
            if recursoAtv not in recursos:
                violacoes["Atividade"].append([atv['concept:name'], demanda, recursoAtv])
            
            acessosAtv = acessos[acessos['concept:instance']== atv['concept:instance']]
            for j, acc in acessosAtv.iterrows():
                if recursoAtv != acc['concept:resource']:
                    violacoes["AcessoRecursoErrado"].append([acc['concept:tool'], demanda, acc['concept:resource'], recursoAtv])
                if acc['concept:resource'] not in recursos:
                    violacoes["AcessoRecursoEquipe"].append([acc['concept:tool'], demanda, acc['concept:resource']])
    return violacoes

In [128]:
def formatInconformances(conformanceProcess, conformanceResource):
    '''
    Formata as inconformidades encontradas em cada verificação (pode vir a combinar os padrões de anomalias no futuro)
    '''
    print('Violações de fluxo de processo:')
    for violation in conformanceProcess:
        print(violation)
    
    # print("Violações de Acesso:")
    # for key, violation in conformanceAcess.items():
    #     if key == "OperacaoProibida":
    #         for occur in violation:
    #             print(f'Violação de Acesso na atividade {occur[0]} no trace {occur[1]}, a operação {occur[2]} não era permitida para a ferramenta {occur[3]}')
    #     if key == "AcessoObgNaoRealizado":
    #         for occur in violation:
    #             print(f"Violação de Acesso na atividade {occur[0]} no trace {occur[1]}, os acessos obrigatórios às ferramentas {occur[2]} não foram realizados")
    
    
    print('Violações de Privacidade:')
    for key, violation in conformanceResource.items():
        if key == "Atividade":
            for occur in violation:
                print(f'Violação de Privacidade na atividade {occur[0]} no trace {occur[1]}, o recurso {occur[2]} não faz parte da equipe determinada para realizar a demanda')
        if key == "AcessoRecursoEquipe":
            for occur in violation:
                print(f'Violação de Privacidade no acesso a {occur[0]} no trace {occur[1]}, o recurso {occur[2]} não faz parte da equipe determinada para realizar a demanda')
        if key == "AcessoRecursoErrado":
            for occur in violation:
                print(f'Violação de Privacidade no acesso a {occur[0]} no trace {occur[1]}, o recurso {occur[2]} não era o designado da atividade vinculada e sim o {occur[3]}')
    

In [129]:
def MultiperspectiveConformanceAlgorithm(logProcessoPATH, logAcessoPATH, modeloRecursoPATH, modeloProcessoPATH, modeloAcessoPATH):
  '''
  O algoritmo recebe: um log de processo, um log de acesso a dados, um modelo de recurso, um modelo DECLARE de processo e um modelo de acesso a dados
  '''
  logProcesso, logAcesso, modeloRecurso, modeloProcesso, modeloAcesso = PreProcessData(logProcessoPATH, logAcessoPATH, modeloRecursoPATH, modeloProcessoPATH, modeloAcessoPATH)
  modeloAcessoConvertido = convertModelToRules(modeloAcesso, modeloProcesso)
  convertLogs(logProcesso, logAcesso)
  conformanceProcess = CheckProcessConformance(modeloAcessoConvertido)
  # conformanceAcess = CheckAcessConformance(logProcesso, logAcesso, modeloAcesso)
  conformanceResource = CheckResourceConformance(logProcesso, logAcesso, modeloRecurso)
  return formatInconformances(conformanceProcess, conformanceResource)

In [130]:
MultiperspectiveConformanceAlgorithm('./LogSinteticoProcessoOFICIALv4.xes', './LogSinteticoAcessoOFICIALv4.xes', './ModeloRecursosOFICIALv4.csv', './Modelo_Log_Sintetico_OFICIAL.decl', './ModeloAcessoOFICIAL.csv')

parsing log, completed traces :: 100%|██████████| 1/1 [00:00<?, ?it/s]


Violações de fluxo de processo:
Trace 0 violou Precedence[Manutencao de funcionalidade begin, Codigo u] | A.concept:resource is re7 | T.concept:resource is re7 |
Trace 0 violou Response[Codigo u, Manutencao de funcionalidade complete] | A.concept:resource is re7 | T.concept:resource is re7 |
Trace 0 violou Response[Requisito u, Atualizacao de requisitos funcionais complete] | A.concept:resource is re7 | T.concept:resource is re7 |
Trace 0 violou Response[Codigo c, Construcao de funcionalidade complete] | A.concept:resource is re7 | T.concept:resource is re7 |
Trace 0 violou Precedence[Documentacao de requisitos funcionais begin, Gestao u] | A.concept:resource is re7 | T.concept:resource is re7 |
Violações de Privacidade:
Violação de Privacidade no acesso a Gestao no trace Demanda 0001, o recurso re10 não faz parte da equipe determinada para realizar a demanda
Violação de Privacidade no acesso a Gestao no trace Demanda 0001, o recurso re10 não era o designado da atividade vinculada e si